# Pulser-MyQLM Conversion

In [ ]:
import numpy as np
import qat
from qat.core import Schedule

# from qat.qpus import AnalogQPU
from qlmaas.qpus import AnalogQPU

# import qse
from time import time
import qse
import numpy as np
import qat
from qat.core import Schedule

# from qat.qpus import AnalogQPU
from qlmaas.qpus import AnalogQPU

# import qse
from time import time
import pickle
import numpy as np
import pulser
import qutip
from pprint import pprint
from pulser import Pulse, Sequence, Register
import pulser.waveforms
from pulser.waveforms import ConstantWaveform, RampWaveform
from pulser.devices import MockDevice
from pulser_simulation import QutipEmulator, SimConfig
import matplotlib.pyplot as plt
import qse
from pulser.devices import AnalogDevice

## Testing a one atom setup with 0 amplitude and detuning

In [ ]:
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0
R_interatomic = MockDevice.rydberg_blockade_radius(U)
reg = Register.square(1, R_interatomic, prefix="q")
print(f"Interatomic Radius is: {R_interatomic}µm.")
reg.draw()

In [ ]:
delta_0 = -6 * U
delta_f = 2 * U
t_rise = 252
t_fall = 500
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000
rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0.0, 0.0), 0, 0.0)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.0)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.0), delta_f, 0.0)
seq1 = Sequence(reg, MockDevice)
seq1.declare_channel("ising", "rydberg_global")

seq1.add(rise, "ising")
seq1.add(sweep, "ising")
seq1.add(fall, "ising")

seq1.draw()

In [ ]:
sim = QutipEmulator.from_sequence(seq1)

In [ ]:
import qse.qbits

L = 3
nqbits = L * L
qsqr = qse.Qbits(positions=np.array([[0, 0, 0]]))


Omega = 1.2  # Amplitude
delta = 2.0  # detuning
C6 = 5420158.53  # interaction scale
nqbits = 1

In [ ]:
H1_terms = [qat.core.Term(0.5, "X", [i]) for i in range(qsqr.nqbits)]
H1 = qat.core.Observable(nqbits, pauli_terms=H1_terms)

H2_terms = [qat.core.Term(-0.5, "Z", [i]) for i in range(qsqr.nqbits)]
H2 = qat.core.Observable(nqbits, pauli_terms=H2_terms)

rij = qsqr.get_all_distances()


def occ_op(nqbits, qi):
    ti = qat.core.Term(1.0, "Z", [qi])
    return (1 + qat.core.Observable(nqbits, pauli_terms=[ti])) / 2


#
# H3 = 0

# for i in range(nqbits):
#     for j in range(i + 1, nqbits):
#         H3 += (C6 / rij[i, j]**6) * occ_op(nqbits, i) * occ_op(nqbits, j)

In [ ]:
from qat.core.variables import Variable, heaviside
from qat.core import Observable


def create_waveform(times, values):
    """
    A Funnction allowing us to concatenate many lines, limited by heavisides,
    thus creating a continuous, kinked line.
    """
    assert len(times) == len(values)
    # Store the final line as an ArithExpression
    arith_expr = 0
    t_var = Variable("t")
    # A line between each two points
    for entry_i in range(1, len(times)):
        # Create ax + b by calculating the slope and the offset
        a = (values[entry_i] - values[entry_i - 1]) / (
            times[entry_i] - times[entry_i - 1]
        )
        b = (
            (values[entry_i] + values[entry_i - 1])
            - a * (times[entry_i] + times[entry_i - 1])
        ) / 2
        respective_line = a * t_var + b
        arith_expr += (
            heaviside(t_var, times[entry_i - 1], times[entry_i]) * respective_line
        )
    return arith_expr


#
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0

delta_0 = -6 * U
delta_f = 2 * U

t_rise = 0.252
t_fall = 0.500
t_sweep = ((delta_f - delta_0) / (2 * np.pi * 10) * 1000) * (10**-3)


# The waveform of Omega - for the sigma_x terms
omega_values = [0.0, 0, Omega_max, 0]
times_omega = [0, t_rise, t_rise + t_sweep, t_rise + t_sweep + t_fall]


omega_waveform = create_waveform(times_omega, omega_values)
# The waveform of the detuning Delta - for the 'n' terms
delta_start = -2 * np.pi * 13
delta_end = 2 * np.pi * 11  # R_blockade is taken as 7.5
delta_values = [0, 0, delta_f, delta_f]
delta_waveform = create_waveform(times_omega, delta_values)

In [ ]:
from qat.core import Observable

Hamiltonian = Observable(
    [
        (omega_waveform, H1),
        (delta_waveform, H2),
    ]
)

schedule = Schedule(
    drive=[
        (omega_waveform, H1),
        (delta_waveform, H2),
    ],
    tmax=t_rise + t_sweep + t_fall,
)

In [ ]:
schedule.display()

In [ ]:
matrixm = 0 * H1.to_matrix(sparse=False) + 0 * H2.to_matrix(sparse=False)

In [ ]:
matrixp = sim.get_hamiltonian(0).get_data().toarray()
plt.imshow(matrixp.real)
print(matrixp)
np.savetxt("matrix-p.dat", matrixp.real)
np.all(matrixp.imag == 0)

In [ ]:
plt.imshow(matrixm.real)
print(matrixm)
np.savetxt("matrix-m.dat", matrixm.real)
np.all(matrixm.imag == 0)

In [ ]:
plt.imshow(matrix0.real)
print(matrix0)
np.savetxt("matrix-q.dat", matrix0.real)
np.all(matrix0.imag == 0)

## Testing a one atom setup with 0 amplitude and detuning set to fixed value

In [ ]:
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0
R_interatomic = MockDevice.rydberg_blockade_radius(U)
reg = Register.square(1, R_interatomic, prefix="q")
print(f"Interatomic Radius is: {R_interatomic}µm.")
reg.draw()

In [ ]:
delta_0 = -6 * U
delta_f = 2 * U
t_rise = 252
t_fall = 500
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000
rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0.0, 0.0), delta_0, 0.0)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.0)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.0), delta_f, 0.0)
seq1 = Sequence(reg, MockDevice)
seq1.declare_channel("ising", "rydberg_global")

seq1.add(rise, "ising")
seq1.add(sweep, "ising")
seq1.add(fall, "ising")

seq1.draw()

In [ ]:
sim = QutipEmulator.from_sequence(seq1)

In [ ]:
import qse.qbits

L = 3
nqbits = L * L
qsqr = qse.Qbits(positions=np.array([[0, 0, 0]]))


Omega = 1.2  # Amplitude
delta = 2.0  # detuning
C6 = 5420158.53  # interaction scale
nqbits = 1

In [ ]:
H1_terms = [qat.core.Term(0.5, "X", [i]) for i in range(qsqr.nqbits)]
H1 = qat.core.Observable(nqbits, pauli_terms=H1_terms)

H2_terms = [qat.core.Term(-0.5, "Z", [i]) for i in range(qsqr.nqbits)]
H2 = qat.core.Observable(nqbits, pauli_terms=H2_terms)

rij = qsqr.get_all_distances()


def occ_op(nqbits, qi):
    ti = qat.core.Term(1.0, "Z", [qi])
    return (1 + qat.core.Observable(nqbits, pauli_terms=[ti])) / 2


#
# H3 = 0

# for i in range(nqbits):
#     for j in range(i + 1, nqbits):
#         H3 += (C6 / rij[i, j]**6) * occ_op(nqbits, i) * occ_op(nqbits, j)

In [ ]:
from qat.core.variables import Variable, heaviside
from qat.core import Observable


def create_waveform(times, values):
    """
    A Funnction allowing us to concatenate many lines, limited by heavisides,
    thus creating a continuous, kinked line.
    """
    assert len(times) == len(values)
    # Store the final line as an ArithExpression
    arith_expr = 0
    t_var = Variable("t")
    # A line between each two points
    for entry_i in range(1, len(times)):
        # Create ax + b by calculating the slope and the offset
        a = (values[entry_i] - values[entry_i - 1]) / (
            times[entry_i] - times[entry_i - 1]
        )
        b = (
            (values[entry_i] + values[entry_i - 1])
            - a * (times[entry_i] + times[entry_i - 1])
        ) / 2
        respective_line = a * t_var + b
        arith_expr += (
            heaviside(t_var, times[entry_i - 1], times[entry_i]) * respective_line
        )
    return arith_expr


#
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0

delta_0 = -6 * U
delta_f = 2 * U

t_rise = 0.252
t_fall = 0.500
t_sweep = ((delta_f - delta_0) / (2 * np.pi * 10) * 1000) * (10**-3)


# The waveform of Omega - for the sigma_x terms
omega_values = [0.0, 0, Omega_max, 0]
times_omega = [0, t_rise, t_rise + t_sweep, t_rise + t_sweep + t_fall]


omega_waveform = create_waveform(times_omega, omega_values)
# The waveform of the detuning Delta - for the 'n' terms
delta_start = -2 * np.pi * 13
delta_end = 2 * np.pi * 11  # R_blockade is taken as 7.5
delta_values = [delta_0, 0, delta_f, delta_f]
delta_waveform = create_waveform(times_omega, delta_values)

In [ ]:
from qat.core import Observable

Hamiltonian = Observable(
    [
        (omega_waveform, H1),
        (delta_waveform, H2),
    ]
)

schedule = Schedule(
    drive=[
        (omega_waveform, H1),
        (delta_waveform, H2),
    ],
    tmax=t_rise + t_sweep + t_fall,
)

In [ ]:
schedule.display()

In [ ]:
matrixm = 0 * H1.to_matrix(sparse=False) + delta_0 * H2.to_matrix(sparse=False)

In [ ]:
matrixp = sim.get_hamiltonian(0).get_data().toarray()
plt.imshow(matrixp.real)
print(matrixp)
np.savetxt("matrix-p.dat", matrixp.real)
np.all(matrixp.imag == 0)

In [ ]:
plt.imshow(matrixm.real)
print(matrixm)
np.savetxt("matrix-m.dat", matrixm.real)
np.all(matrixm.imag == 0)

## Testing one atom with Amplitude set to fixed number and detuning to 0


In [ ]:
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0
R_interatomic = MockDevice.rydberg_blockade_radius(U)
reg = Register.square(1, R_interatomic, prefix="q")
print(f"Interatomic Radius is: {R_interatomic}µm.")
reg.draw()

In [ ]:
delta_0 = -6 * U
delta_f = 2 * U
t_rise = 252
t_fall = 500
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000
rise = Pulse.ConstantDetuning(RampWaveform(t_rise, Omega_max, Omega_max), 0, 0.0)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.0)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.0), delta_f, 0.0)
seq1 = Sequence(reg, MockDevice)
seq1.declare_channel("ising", "rydberg_global")

seq1.add(rise, "ising")
seq1.add(sweep, "ising")
seq1.add(fall, "ising")

seq1.draw()

In [ ]:
sim = QutipEmulator.from_sequence(seq1)

In [ ]:
import qse.qbits

L = 3
nqbits = L * L
qsqr = qse.Qbits(positions=np.array([[0, 0, 0]]))


Omega = 1.2  # Amplitude
delta = 2.0  # detuning
C6 = 5420158.53  # interaction scale
nqbits = 1

In [ ]:
H1_terms = [qat.core.Term(0.5, "X", [i]) for i in range(qsqr.nqbits)]
H1 = qat.core.Observable(nqbits, pauli_terms=H1_terms)

H2_terms = [qat.core.Term(-0.5, "Z", [i]) for i in range(qsqr.nqbits)]
H2 = qat.core.Observable(nqbits, pauli_terms=H2_terms)

rij = qsqr.get_all_distances()


def occ_op(nqbits, qi):
    ti = qat.core.Term(1.0, "Z", [qi])
    return (1 + qat.core.Observable(nqbits, pauli_terms=[ti])) / 2


#
# H3 = 0

# for i in range(nqbits):
#     for j in range(i + 1, nqbits):
#         H3 += (C6 / rij[i, j]**6) * occ_op(nqbits, i) * occ_op(nqbits, j)

In [ ]:
from qat.core.variables import Variable, heaviside
from qat.core import Observable


def create_waveform(times, values):
    """
    A Funnction allowing us to concatenate many lines, limited by heavisides,
    thus creating a continuous, kinked line.
    """
    assert len(times) == len(values)
    # Store the final line as an ArithExpression
    arith_expr = 0
    t_var = Variable("t")
    # A line between each two points
    for entry_i in range(1, len(times)):
        # Create ax + b by calculating the slope and the offset
        a = (values[entry_i] - values[entry_i - 1]) / (
            times[entry_i] - times[entry_i - 1]
        )
        b = (
            (values[entry_i] + values[entry_i - 1])
            - a * (times[entry_i] + times[entry_i - 1])
        ) / 2
        respective_line = a * t_var + b
        arith_expr += (
            heaviside(t_var, times[entry_i - 1], times[entry_i]) * respective_line
        )
    return arith_expr


#
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0

delta_0 = -6 * U
delta_f = 2 * U

t_rise = 0.252
t_fall = 0.500
t_sweep = ((delta_f - delta_0) / (2 * np.pi * 10) * 1000) * (10**-3)


# The waveform of Omega - for the sigma_x terms
omega_values = [Omega_max, 0, Omega_max, 0]
times_omega = [0, t_rise, t_rise + t_sweep, t_rise + t_sweep + t_fall]


omega_waveform = create_waveform(times_omega, omega_values)
# The waveform of the detuning Delta - for the 'n' terms
delta_start = -2 * np.pi * 13
delta_end = 2 * np.pi * 11  # R_blockade is taken as 7.5
delta_values = [0, 0, delta_f, delta_f]
delta_waveform = create_waveform(times_omega, delta_values)

In [ ]:
from qat.core import Observable

Hamiltonian = Observable(
    [
        (omega_waveform, H1),
        (delta_waveform, H2),
    ]
)

schedule = Schedule(
    drive=[
        (omega_waveform, H1),
        (delta_waveform, H2),
    ],
    tmax=t_rise + t_sweep + t_fall,
)

In [ ]:
schedule.display()

In [ ]:
matrixm = Omega_max * H1.to_matrix(sparse=False) + 0 * H2.to_matrix(sparse=False)

In [ ]:
matrixp = sim.get_hamiltonian(0).get_data().toarray()
plt.imshow(matrixp.real)
print(matrixp)
np.savetxt("matrix-p.dat", matrixp.real)

In [ ]:
plt.imshow(matrixm.real)
print(matrixm)
np.savetxt("matrix-m.dat", matrixm.real)
np.all(matrixm.imag == 0)

## Testing a one atom setup with 0 amplitude and detuning checking interaction term

In [ ]:
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0
R_interatomic = MockDevice.rydberg_blockade_radius(U)
reg = Register.from_coordinates([[0, 0], [2, 0]])
print(f"Interatomic Radius is: {R_interatomic}µm.")
reg.draw()

In [ ]:
delta_0 = -6 * U
delta_f = 2 * U
t_rise = 252
t_fall = 500
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000
rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0.0, 0.0), 0, 0.0)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.0)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.0), delta_f, 0.0)
seq1 = Sequence(reg, MockDevice)
seq1.declare_channel("ising", "rydberg_global")

seq1.add(rise, "ising")
seq1.add(sweep, "ising")
seq1.add(fall, "ising")

seq1.draw()

In [ ]:
sim = QutipEmulator.from_sequence(seq1)

In [ ]:
import qse.qbits

L = 3
nqbits = L * L
qsqr = qse.Qbits(positions=np.array([[0, 0, 0], [2, 0, 0]]))


Omega = 1.2  # Amplitude
delta = 2.0  # detuning
C6 = 5420158.53  # interaction scale
nqbits = 2

In [ ]:
H1_terms = [qat.core.Term(0.5, "X", [i]) for i in range(qsqr.nqbits)]
H1 = qat.core.Observable(nqbits, pauli_terms=H1_terms)

H2_terms = [qat.core.Term(-0.5, "Z", [i]) for i in range(qsqr.nqbits)]
H2 = qat.core.Observable(nqbits, pauli_terms=H2_terms)

rij = qsqr.get_all_distances()


def occ_op(nqbits, qi):
    ti = qat.core.Term(1.0, "Z", [qi])
    return (1 + qat.core.Observable(nqbits, pauli_terms=[ti])) / 2


#
H3 = 0

for i in range(nqbits):
    for j in range(i + 1, nqbits):
        H3 += (C6 / rij[i, j] ** 6) * occ_op(nqbits, i) * occ_op(nqbits, j)

In [ ]:
from qat.core.variables import Variable, heaviside
from qat.core import Observable


def create_waveform(times, values):
    """
    A Funnction allowing us to concatenate many lines, limited by heavisides,
    thus creating a continuous, kinked line.
    """
    assert len(times) == len(values)
    # Store the final line as an ArithExpression
    arith_expr = 0
    t_var = Variable("t")
    # A line between each two points
    for entry_i in range(1, len(times)):
        # Create ax + b by calculating the slope and the offset
        a = (values[entry_i] - values[entry_i - 1]) / (
            times[entry_i] - times[entry_i - 1]
        )
        b = (
            (values[entry_i] + values[entry_i - 1])
            - a * (times[entry_i] + times[entry_i - 1])
        ) / 2
        respective_line = a * t_var + b
        arith_expr += (
            heaviside(t_var, times[entry_i - 1], times[entry_i]) * respective_line
        )
    return arith_expr


#
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0

delta_0 = -6 * U
delta_f = 2 * U

t_rise = 0.252
t_fall = 0.500
t_sweep = ((delta_f - delta_0) / (2 * np.pi * 10) * 1000) * (10**-3)


# The waveform of Omega - for the sigma_x terms
omega_values = [0.0, 0, Omega_max, 0]
times_omega = [0, t_rise, t_rise + t_sweep, t_rise + t_sweep + t_fall]


omega_waveform = create_waveform(times_omega, omega_values)
# The waveform of the detuning Delta - for the 'n' terms
delta_start = -2 * np.pi * 13
delta_end = 2 * np.pi * 11  # R_blockade is taken as 7.5
delta_values = [0, 0, delta_f, delta_f]
delta_waveform = create_waveform(times_omega, delta_values)

In [ ]:
from qat.core import Observable

Hamiltonian = Observable(
    [
        (omega_waveform, H1),
        (delta_waveform, H2),
    ]
)

schedule = Schedule(
    drive=[
        (omega_waveform, H1),
        (delta_waveform, H2),
    ],
    tmax=t_rise + t_sweep + t_fall,
)

In [ ]:
schedule.display()

In [ ]:
matrixm = (
    0 * H1.to_matrix(sparse=False)
    + 0 * H2.to_matrix(sparse=False)
    + H3.to_matrix(sparse=False)
)

In [ ]:
H3.to_matrix(sparse=False)

In [ ]:
matrixp = sim.get_hamiltonian(0).get_data().toarray()
plt.imshow(matrixp.real)
print(matrixp)
np.savetxt("matrix-p.dat", matrixp.real)
np.all(matrixp.imag == 0)

In [ ]:
plt.imshow(matrixm.real)
print(matrixm)
np.savetxt("matrix-m.dat", matrixm.real)
np.all(matrixm.imag == 0)

## Testing more atoms with 0 amplitude and detuning checking interaction term

In [ ]:
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0
R_interatomic = MockDevice.rydberg_blockade_radius(U)
reg = Register.from_coordinates(
    [[0, 0], [2, 0], [4, 0], [0, 2], [2, 2], [4, 2], [0, 4], [2, 4], [4, 4]]
)
print(f"Interatomic Radius is: {R_interatomic}µm.")
reg.draw()

In [ ]:
delta_0 = -6 * U
delta_f = 2 * U
t_rise = 252
t_fall = 500
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000
rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0.0, 0.0), 0, 0.0)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.0)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.0), delta_f, 0.0)
seq1 = Sequence(reg, MockDevice)
seq1.declare_channel("ising", "rydberg_global")

seq1.add(rise, "ising")
seq1.add(sweep, "ising")
seq1.add(fall, "ising")

seq1.draw()

In [ ]:
sim = QutipEmulator.from_sequence(seq1)

In [ ]:
import qse.qbits

L = 3
nqbits = L * L
qsqr = qse.Qbits(
    positions=np.array(
        [
            [0, 0, 0],
            [2, 0, 0],
            [4, 0, 0],
            [0, 2, 0],
            [2, 2, 0],
            [4, 2, 0],
            [0, 4, 0],
            [2, 4, 0],
            [4, 4, 0],
        ]
    )
)


Omega = 1.2  # Amplitude
delta = 2.0  # detuning
C6 = 5420158.53  # interaction scale

In [ ]:
H1_terms = [qat.core.Term(0.5, "X", [i]) for i in range(qsqr.nqbits)]
H1 = qat.core.Observable(nqbits, pauli_terms=H1_terms)

H2_terms = [qat.core.Term(-0.5, "Z", [i]) for i in range(qsqr.nqbits)]
H2 = qat.core.Observable(nqbits, pauli_terms=H2_terms)

rij = qsqr.get_all_distances()


def occ_op(nqbits, qi):
    ti = qat.core.Term(1.0, "Z", [qi])
    return (1 + qat.core.Observable(nqbits, pauli_terms=[ti])) / 2


#
H3 = 0

for i in range(nqbits):
    for j in range(i + 1, nqbits):
        H3 += (C6 / rij[i, j] ** 6) * occ_op(nqbits, i) * occ_op(nqbits, j)

In [ ]:
from qat.core.variables import Variable, heaviside
from qat.core import Observable


def create_waveform(times, values):
    """
    A Funnction allowing us to concatenate many lines, limited by heavisides,
    thus creating a continuous, kinked line.
    """
    assert len(times) == len(values)
    # Store the final line as an ArithExpression
    arith_expr = 0
    t_var = Variable("t")
    # A line between each two points
    for entry_i in range(1, len(times)):
        # Create ax + b by calculating the slope and the offset
        a = (values[entry_i] - values[entry_i - 1]) / (
            times[entry_i] - times[entry_i - 1]
        )
        b = (
            (values[entry_i] + values[entry_i - 1])
            - a * (times[entry_i] + times[entry_i - 1])
        ) / 2
        respective_line = a * t_var + b
        arith_expr += (
            heaviside(t_var, times[entry_i - 1], times[entry_i]) * respective_line
        )
    return arith_expr


#
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0

delta_0 = -6 * U
delta_f = 2 * U

t_rise = 0.252
t_fall = 0.500
t_sweep = ((delta_f - delta_0) / (2 * np.pi * 10) * 1000) * (10**-3)


# The waveform of Omega - for the sigma_x terms
omega_values = [0.0, 0, Omega_max, 0]
times_omega = [0, t_rise, t_rise + t_sweep, t_rise + t_sweep + t_fall]


omega_waveform = create_waveform(times_omega, omega_values)
# The waveform of the detuning Delta - for the 'n' terms
delta_start = -2 * np.pi * 13
delta_end = 2 * np.pi * 11  # R_blockade is taken as 7.5
delta_values = [0, 0, delta_f, delta_f]
delta_waveform = create_waveform(times_omega, delta_values)

In [ ]:
from qat.core import Observable

Hamiltonian = Observable(
    [
        (omega_waveform, H1),
        (delta_waveform, H2),
    ]
)

schedule = Schedule(
    drive=[
        (omega_waveform, H1),
        (delta_waveform, H2),
    ],
    tmax=t_rise + t_sweep + t_fall,
)

In [ ]:
schedule.display()

In [ ]:
matrixm = (
    0 * H1.to_matrix(sparse=False)
    + 0 * H2.to_matrix(sparse=False)
    + H3.to_matrix(sparse=False)
)

In [ ]:
H3.to_matrix(sparse=False)

In [ ]:
matrixp = sim.get_hamiltonian(0).get_data().toarray()
plt.imshow(matrixp.real)
print(matrixp)
np.savetxt("matrix-p.dat", matrixp.real)
np.all(matrixp.imag == 0)

In [ ]:
plt.imshow(matrixm.real)
print(matrixm)
np.savetxt("matrix-m.dat", matrixm.real)
np.all(matrixm.imag == 0)

In [ ]:
np.savetxt("matrix-q.dat", matrixm.real)
np.savetxt("matrix-p.dat", matrixp.real)

In [ ]:
p = np.loadtxt("matrix-p.dat")
q = np.loadtxt("matrix-q.dat")
plt.imshow(p == q)

pd = p.diagonal()
qd = q.diagonal()
print(p / q)

In [ ]:
print(pd / qd)

In [ ]:
print(qd)

## Testing more atoms with fixed amplitude and 0 detuning checking interaction term

In [ ]:
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0
R_interatomic = MockDevice.rydberg_blockade_radius(U)
reg = Register.from_coordinates(
    [[0, 0], [10, 0], [20, 0], [0, 10], [10, 10], [20, 10], [0, 20], [10, 20], [20, 20]]
)
print(f"Interatomic Radius is: {R_interatomic}µm.")
reg.draw()

In [ ]:
delta_0 = -6 * U
delta_f = 2 * U
t_rise = 252
t_fall = 500
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000
rise = Pulse.ConstantAmplitude(10, RampWaveform(t_rise, 0.0, 0.0), 0.0)
sweep = Pulse.ConstantAmplitude(10, RampWaveform(t_sweep, delta_0, delta_f), 0.0)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.0), delta_f, 0.0)
seq1 = Sequence(reg, MockDevice)
seq1.declare_channel("ising", "rydberg_global")

seq1.add(rise, "ising")
seq1.add(sweep, "ising")
seq1.add(fall, "ising")

seq1.draw()

In [ ]:
sim = QutipEmulator.from_sequence(seq1)

In [ ]:
results = sim.run(progress_bar=True)

In [ ]:
import matplotlib.pyplot as plt

count = results.sample_final_state()

most_freq = {k: v for k, v in count.items() if v > 10}
plt.bar(list(most_freq.keys()), list(most_freq.values()))
plt.xticks(rotation="vertical")
plt.show()

In [ ]:
# Define Rydberg state observable
obs = qutip.basis(256 * 2, 0).proj()
plt.figure(figsize=(6, 6))
k = results.expect([obs])[0]

plt.plot(results._sim_times, k)
plt.xlabel(r"Time ($\mu s$)")
plt.ylabel(r"Population in $|r\rangle$")
plt.show()

In [ ]:
import qse.qbits

L = 3
nqbits = L * L
qsqr = qse.Qbits(
    positions=np.array(
        [
            [0, 0, 0],
            [10, 0, 0],
            [20, 0, 0],
            [0, 10, 0],
            [10, 10, 0],
            [20, 10, 0],
            [0, 20, 0],
            [10, 20, 0],
            [20, 20, 0],
        ]
    )
)


Omega = 1.2  # Amplitude
delta = 2.0  # detuning
C6 = 5420158.53  # interaction scale

qsqr.draw()

In [ ]:
H1_terms = [qat.core.Term(0.5, "X", [i]) for i in range(qsqr.nqbits)]
H1 = qat.core.Observable(nqbits, pauli_terms=H1_terms)

H2_terms = [qat.core.Term(-0.5, "Z", [i]) for i in range(qsqr.nqbits)]
H2 = qat.core.Observable(nqbits, pauli_terms=H2_terms)

rij = qsqr.get_all_distances()


def occ_op(nqbits, qi):
    ti = qat.core.Term(1.0, "Z", [qi])
    return (1 + qat.core.Observable(nqbits, pauli_terms=[ti])) / 2


#
H3 = 0

for i in range(nqbits):
    for j in range(i + 1, nqbits):
        H3 += (C6 / rij[i, j] ** 6) * occ_op(nqbits, i) * occ_op(nqbits, j)

In [ ]:
from qat.core.variables import Variable, heaviside
from qat.core import Observable


def create_waveform(times, values):
    """
    A Funnction allowing us to concatenate many lines, limited by heavisides,
    thus creating a continuous, kinked line.
    """
    assert len(times) == len(values)
    # Store the final line as an ArithExpression
    arith_expr = 0
    t_var = Variable("t")
    # A line between each two points
    for entry_i in range(1, len(times)):
        # Create ax + b by calculating the slope and the offset
        a = (values[entry_i] - values[entry_i - 1]) / (
            times[entry_i] - times[entry_i - 1]
        )
        b = (
            (values[entry_i] + values[entry_i - 1])
            - a * (times[entry_i] + times[entry_i - 1])
        ) / 2
        respective_line = a * t_var + b
        arith_expr += (
            heaviside(t_var, times[entry_i - 1], times[entry_i]) * respective_line
        )
    return arith_expr


#
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0

delta_0 = -6 * U
delta_f = 2 * U

t_rise = 0.252
t_fall = 0.500
t_sweep = ((delta_f - delta_0) / (2 * np.pi * 10) * 1000) * (10**-3)


# The waveform of Omega - for the sigma_x terms
omega_values = [10.0, 0, Omega_max, 0]
times_omega = [0, t_rise, t_rise + t_sweep, t_rise + t_sweep + t_fall]


omega_waveform = create_waveform(times_omega, omega_values)
# The waveform of the detuning Delta - for the 'n' terms
delta_start = -2 * np.pi * 13
delta_end = 2 * np.pi * 11  # R_blockade is taken as 7.5
delta_values = [0, 0, delta_f, delta_f]
delta_waveform = create_waveform(times_omega, delta_values)

In [ ]:
from qat.core import Observable

Hamiltonian = Observable(
    [
        (omega_waveform, H1),
        (delta_waveform, H2),
    ]
)

schedule = Schedule(
    drive=[
        (omega_waveform, H1),
        (delta_waveform, H2),
    ],
    tmax=t_rise + t_sweep + t_fall,
)

In [ ]:
schedule.display()

In [ ]:
matrixm = (
    10 * H1.to_matrix(sparse=False)
    + 0 * H2.to_matrix(sparse=False)
    + H3.to_matrix(sparse=False)
)

In [ ]:
H3.to_matrix(sparse=False)

In [ ]:
matrixp = sim.get_hamiltonian(0).get_data().toarray()
plt.imshow(matrixp.real)
print(matrixp)
np.savetxt("matrix-p.dat", matrixp.real)
np.all(matrixp.imag == 0)

In [ ]:
plt.imshow(matrixm.real)
print(matrixm)
np.savetxt("matrix-m.dat", matrixm.real)
np.all(matrixm.imag == 0)

In [ ]:
np.savetxt("matrix-q.dat", matrixm.real)
np.savetxt("matrix-p.dat", matrixp.real)

In [ ]:
p = np.loadtxt("matrix-p.dat")
q = np.loadtxt("matrix-q.dat")
plt.imshow(p == q)

pd = p.diagonal()
qd = q.diagonal()
print(p / q)

In [ ]:
print(pd / qd)

In [ ]:
print(qd)

## Testing more atoms with fixed amplitude and detuning Pauli term fixed checking interaction term

In [ ]:
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0
R_interatomic = MockDevice.rydberg_blockade_radius(U)
reg = Register.from_coordinates(
    [[0, 0], [10, 0], [20, 0], [0, 10], [10, 10], [20, 10], [0, 20], [10, 20], [20, 20]]
)
print(f"Interatomic Radius is: {R_interatomic}µm.")
reg.draw()

In [ ]:
delta_0 = -6 * U
delta_f = 2 * U
t_rise = 252
t_fall = 500
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000
rise = Pulse.ConstantAmplitude(10, RampWaveform(t_rise, 10.0, 0.0), 0)
sweep = Pulse.ConstantAmplitude(10, RampWaveform(t_sweep, delta_0, delta_f), 0)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.0), delta_f, 0.0)
seq1 = Sequence(reg, MockDevice)
seq1.declare_channel("ising", "rydberg_global")

seq1.add(rise, "ising")
seq1.add(sweep, "ising")
seq1.add(fall, "ising")

seq1.draw()

In [ ]:
sim = QutipEmulator.from_sequence(seq1)

In [ ]:
results = sim.run(progress_bar=True)

In [ ]:
import matplotlib.pyplot as plt

count = results.sample_final_state()

most_freq = {k: v for k, v in count.items() if v > 10}
plt.bar(list(most_freq.keys()), list(most_freq.values()))
plt.xticks(rotation="vertical")
plt.show()

In [ ]:
# Define Rydberg state observable
obs = qutip.basis(256 * 2, 0).proj()
plt.figure(figsize=(6, 6))
k = results.expect([obs])[0]

plt.plot(results._sim_times, k)
plt.xlabel(r"Time ($\mu s$)")
plt.ylabel(r"Population in $|r\rangle$")
plt.show()

In [ ]:
import qse.qbits

L = 3
nqbits = L * L
qsqr = qse.Qbits(
    positions=np.array(
        [
            [0, 0, 0],
            [10, 0, 0],
            [20, 0, 0],
            [0, 10, 0],
            [10, 10, 0],
            [20, 10, 0],
            [0, 20, 0],
            [10, 20, 0],
            [20, 20, 0],
        ]
    )
)


Omega = 1.2  # Amplitude
delta = 2.0  # detuning
C6 = 5420158.53  # interaction scale

qsqr.draw()

In [ ]:
def occ_op(nqbits, qi):
    ti = qat.core.Term(1.0, "Z", [qi])
    return (1 + qat.core.Observable(nqbits, pauli_terms=[ti])) / 2


#
H1_terms = [qat.core.Term(0.5, "X", [i]) for i in range(qsqr.nqbits)]
H1 = qat.core.Observable(nqbits, pauli_terms=H1_terms)

# H2_terms = [qat.core.Term(-0.5, "Z", [i])+qat.core.Term(-0.5,"I",[i]) for i in range(qsqr.nqbits)]
# # H2_terms_Z = [qat.core.Term(-0.5, "Z", [i]) for i in range(qsqr.nqbits)]
# H2 = qat.core.Observable(nqbits, pauli_terms=H2_terms)
H2 = 0
for i in range(qsqr.nqbits):
    H2 += occ_op(qsqr.nqbits, i)
H2 = H2 * (-1)
rij = qsqr.get_all_distances()


H3 = 0

for i in range(nqbits):
    for j in range(i + 1, nqbits):
        H3 += (C6 / rij[i, j] ** 6) * occ_op(nqbits, i) * occ_op(nqbits, j)

In [ ]:
from qat.core.variables import Variable, heaviside
from qat.core import Observable


def create_waveform(times, values):
    """
    A Funnction allowing us to concatenate many lines, limited by heavisides,
    thus creating a continuous, kinked line.
    """
    assert len(times) == len(values)
    # Store the final line as an ArithExpression
    arith_expr = 0
    t_var = Variable("t")
    # A line between each two points
    for entry_i in range(1, len(times)):
        # Create ax + b by calculating the slope and the offset
        a = (values[entry_i] - values[entry_i - 1]) / (
            times[entry_i] - times[entry_i - 1]
        )
        b = (
            (values[entry_i] + values[entry_i - 1])
            - a * (times[entry_i] + times[entry_i - 1])
        ) / 2
        respective_line = a * t_var + b
        arith_expr += (
            heaviside(t_var, times[entry_i - 1], times[entry_i]) * respective_line
        )
    return arith_expr


#
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0

delta_0 = -6 * U
delta_f = 2 * U

t_rise = 0.252
t_fall = 0.500
t_sweep = ((delta_f - delta_0) / (2 * np.pi * 10) * 1000) * (10**-3)


# The waveform of Omega - for the sigma_x terms
omega_values = [10.0, 0, Omega_max, 0]
times_omega = [0, t_rise, t_rise + t_sweep, t_rise + t_sweep + t_fall]


omega_waveform = create_waveform(times_omega, omega_values)
# The waveform of the detuning Delta - for the 'n' terms
delta_start = -2 * np.pi * 13
delta_end = 2 * np.pi * 11  # R_blockade is taken as 7.5
delta_values = [10, 0, delta_f, delta_f]
delta_waveform = create_waveform(times_omega, delta_values)

In [ ]:
from qat.core import Observable

Hamiltonian = Observable(
    [
        (omega_waveform, H1),
        (delta_waveform, H2),
    ]
)

schedule = Schedule(
    drive=[
        (omega_waveform, H1),
        (delta_waveform, H2),
    ],
    tmax=t_rise + t_sweep + t_fall,
)

In [ ]:
schedule.display()

In [ ]:
matrixm = (
    10 * H1.to_matrix(sparse=False)
    + 10 * H2.to_matrix(sparse=False)
    + H3.to_matrix(sparse=False)
)

In [ ]:
H2.to_matrix(sparse=False)

In [ ]:
matrixp = sim.get_hamiltonian(0).get_data().toarray()
plt.imshow(matrixp.real)
print(matrixp)
np.savetxt("matrix-p.dat", matrixp.real)
np.all(matrixp.imag == 0)

In [ ]:
plt.imshow(matrixm.real)
print(matrixm)
np.savetxt("matrix-m.dat", matrixm.real)
np.all(matrixm.imag == 0)

In [ ]:
np.savetxt("matrix-q.dat", matrixm.real)
np.savetxt("matrix-p.dat", matrixp.real)

In [ ]:
p = np.loadtxt("matrix-p.dat")
q = np.loadtxt("matrix-q.dat")
plt.imshow(p == q)

pd = p.diagonal()
qd = q.diagonal()
print(p / q)

In [ ]:
print(pd / qd)

In [ ]:
print(q)

In [ ]:
print(p)

In [ ]:
def occ_op(nqbits, qi):
    ti = qat.core.Term(1.0, "Z", [qi])
    return (1 + qat.core.Observable(nqbits, pauli_terms=[ti])) / 2

In [ ]:
qubits = qse.Qbits(positions=np.array([[0, 0, 0]]))
H2 = 0
for i in range(qubits.nqbits):
    H2 += occ_op(qubits.nqbits, i)
H2 = H2 * (-1)
rij = qubits.get_all_distances()

In [ ]:
H2.to_matrix(sparse=False)

## Version

In [ ]:
qse.utils.print_environment()